## Project Data Wrangling - We rate dogs

This project will assure you have mastered the subjects covered in the statistics lessons.  The hope is to have this project be as comprehensive of these topics as possible.  Good luck!

## Table of Contents
- [Gathering Data](#gather)
- [Assessing Data](#assess)
- [Cleaning Data](#clean)
- [Analyzing Data](#analyze)




In [31]:
import tweepy
import pandas as pd
import numpy as np
import requests
import os
import json
import re
# use module dotenv to manage API keys and secrets
%load_ext dotenv
%dotenv

consumer_key = os.environ.get('CONSUMER_KEY')
consumer_secret = os.environ.get('CONSUMER_SECRET')
access_token = os.environ.get('ACCESS_TOKEN')
access_secret = os.environ.get('ACCESS_SECRET')
tsv_url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
tsv_filename = "image-predictions.tsv"
json_filename = "tweets_json.txt"


auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


<a id='gather'></a>
### Gathering Data
We gather data from 3 several sources:
1. enhanced Twitter archive: a csv file 'twitter-archive-enhanced.csv' (data stored in `df_tweets_raw`)
2. Additional Data via the Twitter API
3. Image Predictions File

##### 1.) Gathering Data: enhanced twitter archive
Let's start with getting the data from twitter archive file and take a look at a few records:


In [3]:
df_tweets_raw = pd.read_csv('twitter-archive-enhanced.csv')
df_tweets_raw.sample(3)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1751,678991772295516161,NaN,NaN,2015-12-21 17:33:48 +0000,"<a href=""http://twitter.com/download/iphone"" r...",If your Monday isn't going so well just take a...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/678991772...,12,10,None,None,None,None,None
1943,673709992831262724,NaN,NaN,2015-12-07 03:45:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",I know a lot of you are studying for finals. G...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/673709992...,12,10,None,None,None,None,None
227,848324959059550208,NaN,NaN,2017-04-02 00:03:26 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Odin. He's supposed to be giving directio...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/848324959...,12,10,Odin,None,None,None,None


##### 2.) Gathering Data: Additional data via twitter api
Get data via twitter api if not already downloaded

In [44]:
if not (os.path.exists(json_filename)):
    # get tweets via api and store in json list:
    json_list = []
    for tweet_id in df_tweets_raw.tweet_id.head(10):
        tweet = api.get_status(tweet_id, tweet_mode='extended')
        json_list.append(tweet._json)
    #store json list in json file:
    with open(json_filename, 'w') as f:
        for json_tweet in json_list:
            json.dump(json_tweet, f)
    

##### 3.) Gathering Data: Image Prediction file
Get file from internet if not already downloaded and show a few records

In [39]:
if not (os.path.exists(tsv_filename)):
    r = requests.get(tsv_url, allow_redirects=True)
    with open(tsv_filename, 'wb') as f:
            f.write(r.content)
df_img_pred = pd.read_csv(tsv_filename, sep="\t")
df_img_pred.head(10)
df_img_pred[df_img_pred.p1_dog == False]

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
17,666104133288665088,https://pbs.twimg.com/media/CT56LSZWoAAlJj2.jpg,1,hen,0.965932,False,cock,0.033919,False,partridge,0.000052,False
18,666268910803644416,https://pbs.twimg.com/media/CT8QCd1WEAADXws.jpg,1,desktop_computer,0.086502,False,desk,0.085547,False,bookcase,0.079480,False
21,666293911632134144,https://pbs.twimg.com/media/CT8mx7KW4AEQu8N.jpg,1,three-toed_sloth,0.914671,False,otter,0.015250,False,great_grey_owl,0.013207,False
...,...,...,...,...,...,...,...,...,...,...,...,...
2026,882045870035918850,https://pbs.twimg.com/media/DD2oCl2WAAEI_4a.jpg,1,web_site,0.949591,False,dhole,0.017326,False,golden_retriever,0.006941,True
2046,886680336477933568,https://pbs.twimg.com/media/DE4fEDzWAAAyHMM.jpg,1,convertible,0.738995,False,sports_car,0.139952,False,car_wheel,0.044173,False
2052,887517139158093824,https://pbs.twimg.com/ext_tw_video_thumb/88751...,1,limousine,0.130432,False,tow_truck,0.029175,False,shopping_cart,0.026321,False
2071,891689557279858688,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False


![title](https://pbs.twimg.com/media/CT8mx7KW4AEQu8N.jpg)

<a id='assess'></a>
## Assessing Data

### 1.) Advanced Twitter Archive
**Quality:**  
- `name` column: some entries seem to be regular words rather than names (visually explored)
- `rating_denominator` column: several entries are <> 10, indicating invalid rating values (visually explored)


In [6]:
df_tweets_raw.isnull().sum() / df_tweets_raw.shape[0]

tweet_id                      0.000000
in_reply_to_status_id         0.966893
in_reply_to_user_id           0.966893
timestamp                     0.000000
source                        0.000000
text                          0.000000
retweeted_status_id           0.923175
retweeted_status_user_id      0.923175
retweeted_status_timestamp    0.923175
expanded_urls                 0.025042
rating_numerator              0.000000
rating_denominator            0.000000
name                          0.000000
doggo                         0.000000
floofer                       0.000000
pupper                        0.000000
puppo                         0.000000
dtype: float64

In [15]:
df_tweets_raw['doggo'].isnull()

0       False
1       False
2       False
3       False
4       False
        ...  
2351    False
2352    False
2353    False
2354    False
2355    False
Name: doggo, Length: 2356, dtype: bool

In [18]:
df_tweets_raw['doggo'].replace('None', np.nan, inplace=True)

In [27]:
df_tweets_raw[df_tweets_raw.doggo.notna()].doggo

array(['doggo'], dtype=object)

<a id="clean"></a>
## Cleaning Data



In [80]:
#make copy to work with while cleaning the data
df_tweets_clean = df_tweets_raw.copy()

### Tidiness
#### Validity
##### Issue: Some names in the name column aren't actually names but regular words. 
##### Define
Issue is probably due to a naive assumptions in parsing process of the tweet's text ("This is *dogname*"). So, my solution is to create
a new name column and extract only valid names from the name column. That is, only names with more than one letter starting with upper case.
##### Code

In [72]:
df_tweets_clean['name_extract']=''
df_tweets_clean['name_extract']= df_tweets_clean.name.str.extract(r"^([A-Z]\w+)")

##### Test

In [90]:
# check to see if only regluar words haven't been extracted from the original name column
df_tweets_clean[df_tweets_clean.name_extract != df_tweets_clean.name]['name'].value_counts()

a               55
the              8
an               7
very             5
quite            4
one              4
just             4
not              2
mad              2
getting          2
actually         2
O                1
his              1
incredibly       1
this             1
by               1
all              1
light            1
old              1
life             1
space            1
infuriating      1
such             1
my               1
unacceptable     1
officially       1
Name: name, dtype: int64

,name_extract,name,text
22,NaN,such,I've yet to rate a Venezuelan Hover Wiener. Th...
56,NaN,a,Here is a pupper approaching maximum borkdrive...
118,NaN,quite,RT @dog_rates: We only rate dogs. This is quit...
169,NaN,quite,We only rate dogs. This is quite clearly a smo...
193,NaN,quite,"Guys, we only rate dogs. This is quite clearly..."
...,...,...,...
2349,NaN,an,This is an odd dog. Hard on the outside but lo...
2350,NaN,a,This is a truly beautiful English Wilson Staff...
2352,NaN,a,This is a purebred Piers Morgan. Loves to Netf...
2353,NaN,a,Here is a very happy pup. Big fan of well-main...
